# The Diet Problem - Dual

The goal of the diet problem is to find the cheapest combination of foods that will satisfy all the daily nutritional requirements of a person.
The diet problem is one of the first optimization problems to be studied back in the 1930's and 40's. It was first motivated by the Army desire to meet the nutritional requirements of the soldier while minimizing the cost. 

One interesting interpretation of the dual problem (see R.Vanderbei for formal definition of dual) is finding equilibrium prices of nutritional pills given prices and nutritional content of natural foods. However in the modern world many people prefer meal replacement drinks to plain single nutrient pills. In our model we maximize revenue from selling this hypotetical cocktail that is made to repalce daily meal.


# Mathematical Model for the Dual Problem


The problem is formulated as a linear program where the objective is to maximize revenue from selling nutritional drink (daily meal repalcement). Input data is the same as for the  primal diet problem.

Let us introduce the following notations:

m-number of nutrients, 
n-number of foods
C - vector of food unit cost
$$c=(c_1,..c_n)$$
matrix A - amount of nutrient j in food i
$$A=(a_{ij})$$ 
b-vector of daily nutritional requirements
$$b=(b_1,..b_m)$$
U - decision vector ( unit prices of nutrients in the artificially made nutritional drink)
$$U=(u_1,..u_m)$$ 

Then $\sum_{j=1}^m{b_j u_j}$ is a cost of artificial nutritional drink

$\sum_{j=1}^m{a_{ji} u_j}$ is the cost of nutrients in food i

the optimization problem is formulated as follows:

objective - maximize revenue from selling artificial nutritional drink:
$$\max_u(\sum_{j=1}^m{b_j u_j})$$

constraints: prices of nutrients from artificial sources should not exceed prices of natural foods.
$$\sum_{j=1}^m{a_{ji} u_j}<=c_i, i=1,n$$
bounds on decision variables: unit prices of nutritional elements should not be negative
$$u_j>=0, j=1,m$$





The above optimization problem ( linear programming problem) can be solved with Python linprog solver from Scipy module (scipy.optimize.linprog). This is a description of linprog functionality:

Linear programming: minimize a linear objective function subject to linear equality and inequality constraints.

scipy.optimize.linprog(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, bounds=None, method='highs', callback=None, options=None, x0=None, integrality=None

this is a link to full description in scipy documentatation:
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html

We create a python script that loads input data from Excel spreadsheet, transforms it to a form suitable for scipy.optimize.linprog solver, runs scipy.optimize.linprog solver and prints solver output



# Python script

we use pandas read_excel function to read input data

In [2]:
import pandas as pd

we use numpy to perform vector and matrix operations 

In [3]:
import numpy as np

In [4]:
import scipy.optimize as spo

this is a spreadsheet with input data

In [5]:
data_file = 'DietDataFormatted.xlsx'

From this input data we need to prepare matrices and vectors (A,b,c) for linprog input.

First, read nutritional info for each food that we want to be in our daily diet.

In [6]:
df_A = pd.read_excel(data_file,sheet_name='NutritionalInfo',header = 0,index_col = 0)

In [7]:
df_A

,fat,vitA,vitC,protein,calories
broccoli,0.8,70.0,160.2,8.0,74.0
apple,0.5,73.1,7.9,0.3,81.4
oatm.cookie,3.3,2.9,0.1,1.1,81.0
milk,4.7,100.0,2.3,8.1,121.2
chcken,10.8,77.4,2.0,42.2,227.2
omelette,7.3,409.2,0.1,6.7,99.6


convert this DataFrame into numpy array A 

In [8]:
A = df_A.to_numpy() # we transpose  matrix used in primal problem: np.transpose(df_A.to_numpy())

In [47]:
A

array([[8.000e-01, 5.000e-01, 3.300e+00, 4.700e+00, 1.080e+01, 7.300e+00],
       [7.000e+01, 7.310e+01, 2.900e+00, 1.000e+02, 7.740e+01, 4.092e+02],
       [1.602e+02, 7.900e+00, 1.000e-01, 2.300e+00, 2.000e+00, 1.000e-01],
       [8.000e+00, 3.000e-01, 1.100e+00, 8.100e+00, 4.220e+01, 6.700e+00],
       [7.400e+01, 8.140e+01, 8.100e+01, 1.212e+02, 2.272e+02, 9.960e+01]])

read daily requirements info for each nutrient

In [10]:
df_b = pd.read_excel(data_file,sheet_name='Requirements',header = 0,index_col = 0)

In [11]:
df_b

,fat,vitA,vitC,protein,calories
lower bounds,20,50,50,50,1000
upper bounds,65,600,300,110,2000


In [16]:
b = -df_b.to_numpy()[1]


In [17]:
b

array([  -65,  -600,  -300,  -110, -2000], dtype=int64)

read food unit costs

In [18]:
df_c = pd.read_excel(data_file,sheet_name='UnitCosts',header = None,index_col = 0)

In [19]:
df_c

,1
0,
broccoli,0.16
apple,0.24
oatm.cookie,0.09
milk,0.23
chcken,0.84
omelette,0.11


create numpy array from unit costs data

In [20]:
c = df_c.to_numpy().flatten()

In [21]:
c

array([0.16, 0.24, 0.09, 0.23, 0.84, 0.11])

In [43]:
bound = (0, 100)
m = b.shape[0]
bounds = ((bound, ) * m)

In [44]:
bounds

((0, 100), (0, 100), (0, 100), (0, 100), (0, 100))

input matrices and vectors A,b,c are ready, we can run linprog solver

In [45]:
res=spo.linprog(b,A_ub=A,b_ub=c,bounds=bounds)

In [46]:
res

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -2.3545006056988362
              x: [ 0.000e+00  0.000e+00  4.888e-04  0.000e+00  1.104e-03]
            nit: 2
          lower:  residual: [ 0.000e+00  0.000e+00  4.888e-04  0.000e+00
                              1.104e-03]
                 marginals: [ 7.298e+01  7.181e+03  0.000e+00  3.016e+01
                              0.000e+00]
          upper:  residual: [ 1.000e+02  1.000e+02  1.000e+02  1.000e+02
                              1.000e+02]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  1.463e-01  5.330e-04  9.508e-02
                              5.882e-01  0.000e+00]
                 marginals: [-1.861e+00 -0.000e+00 -0.000e+00 -0.000e+00
                             -0.

from the result object we print daily meal overview as follows

In [47]:
print ('**********************************************************')   
print ('cost of daily meal, optimized:', res.fun)
print ('**********************************************************')   
print ('here is the meal plan ( number of servings of each food):')
for i in range(len(df_A.index)):
    print (df_A.index[i], ':', res.x[i])

print ('**********************************************************')    
print ('you are getting these nutrients:')    
for j in range(len(df_A.columns)):
    nutrient_j = 0.0
    for i in range(len(df_A.index)):
        nutrient_j += A[j,i]*res.x[i]
    print (df_A.columns[j] , ':', nutrient_j)    

**********************************************************
cost of daily meal, optimized: -2.3545006056988362
**********************************************************
here is the meal plan ( number of servings of each food):
broccoli : 0.0
apple : 0.0
oatm.cookie : 0.0004888227873181963
milk : 0.0
chcken : 0.0011039268847516887


IndexError: index 5 is out of bounds for axis 0 with size 5